In [1]:
import os, tarfile
import re, glob
import pandas as pd
import numpy as np
import zipfile, gzip
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [2]:
df_train = pd.read_csv("/data/private/pdutta/DNABERT_data/prom-300/prom-300/TATA_249to50.tsv", sep="\t")

In [3]:
df_train

,seq,label,TATA
0,CTCCACTTTTTCTCACGTTTATCTGAGCGAAAACAAGCACGGTTCG...,1,1
1,CCAGCAGATGGAAAACAGGACAATGTAACACTGTTCTTATCATCAC...,1,1
2,TCAGAGAAACTGGTCTCTTGATAATAGCCATAGATTACATACTGTG...,1,1
3,GAAAAGGTAAAAATACAAGTATGGCTGTTGTTGCTTCAGCCGGGCC...,1,1
4,CAGGGTCTGCAGAGAGAGAGTGGGGGAGTGCGGGTCGGAGCTTCTG...,1,1
...,...,...,...
6125,TCGAATGAACCCTGGTAACTAGGGAATGTGCTCGCCCTGTGTCTGA...,0,1
6126,AGGCCAGACCCTCCCACTGGGAGGTATAGCCCAAGCCCAGGAGCTG...,0,1
6127,GCTTCAGGGCTGAATGGATGCTGCAGATGCGGGGCCAGGGGGCCGG...,0,1
6128,AGGCGGCCGCCACCATGCCCGGCTAATTTTTTGTATTTTTAGTAGA...,0,1


In [4]:
def seq2kmer(seq, k):
    """
    Convert original sequence to kmers
    
    Arguments:
    seq -- str, original sequence.
    k -- int, kmer of length k specified.
    
    Returns:
    kmers -- str, kmers separated by space
    """
    kmer = [seq[x:x+k] for x in range(len(seq)+1-k)]
    kmers = " ".join(kmer)
    return kmers

In [5]:
def core_prom_35to34(text):
    return text[214:284]

def core_prom_45to44(text):
    return text[204:294]

In [6]:
df_train['CoreProm-35to34']= df_train['seq'].apply(core_prom_35to34)
df_train['CoreProm-45to44']= df_train['seq'].apply(core_prom_45to44)

In [7]:
df_train

,seq,label,TATA,CoreProm-35to34,CoreProm-45to44
0,CTCCACTTTTTCTCACGTTTATCTGAGCGAAAACAAGCACGGTTCG...,1,1,ACGGGATAAAGTTCCCGGAGAAAGGAAAGGAGAGCGTGGGATAGTA...,CGGCAGGGGAACGGGATAAAGTTCCCGGAGAAAGGAAAGGAGAGCG...
1,CCAGCAGATGGAAAACAGGACAATGTAACACTGTTCTTATCATCAC...,1,1,CAGACCTATAAATCCAGGCTGGCTCCTCACTCCCCACACATCTGCT...,CTTCTCCCAGCAGACCTATAAATCCAGGCTGGCTCCTCACTCCCCA...
2,TCAGAGAAACTGGTCTCTTGATAATAGCCATAGATTACATACTGTG...,1,1,CTTCCTATAAAAGGCAGAGACAGAGCTTCCAGAGGAGCAGAGGGGC...,ATCCTCTCTGCTTCCTATAAAAGGCAGAGACAGAGCTTCCAGAGGA...
3,GAAAAGGTAAAAATACAAGTATGGCTGTTGTTGCTTCAGCCGGGCC...,1,1,TGGTCCCATAAATGGATGTAAAAAGGGGAAGCTTCGAGCCAATTTC...,GGCTGACAGATGGTCCCATAAATGGATGTAAAAAGGGGAAGCTTCG...
4,CAGGGTCTGCAGAGAGAGAGTGGGGGAGTGCGGGTCGGAGCTTCTG...,1,1,CGCGGGCATAAAAGGCGCCAGGTGAGGGCCTCGCCGCTCCTCCCGC...,CTCCACCGGCCGCGGGCATAAAAGGCGCCAGGTGAGGGCCTCGCCG...
...,...,...,...,...,...
6125,TCGAATGAACCCTGGTAACTAGGGAATGTGCTCGCCCTGTGTCTGA...,0,1,TTGGTTCAAAAATGCCGAATTTCTACGTGAGCTCAAGTCAGGCCGC...,GTGCTGTCTCTTGGTTCAAAAATGCCGAATTTCTACGTGAGCTCAA...
6126,AGGCCAGACCCTCCCACTGGGAGGTATAGCCCAAGCCCAGGAGCTG...,0,1,CACTTACAAATGGGTCTGCCAGTGGATTTTAACCTATTTTGCCTGG...,GCACTCGGAGCACTTACAAATGGGTCTGCCAGTGGATTTTAACCTA...
6127,GCTTCAGGGCTGAATGGATGCTGCAGATGCGGGGCCAGGGGGCCGG...,0,1,TGGGGATATGAGAGGCCAGCTGCGAGGAGGGGAGGAGGTAAGGGCT...,GGTTGGGAGGTGGGGATATGAGAGGCCAGCTGCGAGGAGGGGAGGA...
6128,AGGCGGCCGCCACCATGCCCGGCTAATTTTTTGTATTTTTAGTAGA...,0,1,ATCCTATATTAAGCAGTATAATTGAAAAGAAAAAGAAAACAATAGC...,GTGCCACAAAATCCTATATTAAGCAGTATAATTGAAAAGAAAAAGA...


In [8]:
df_Prom_TATA= pd.DataFrame()
df_coreProm70_TATA =pd.DataFrame()
df_coreProm90_TATA =pd.DataFrame()

In [9]:
df_Prom_TATA['Sequence'] = df_train['seq'].apply(seq2kmer, args=(6,))
df_Prom_TATA['Label'] = df_train['label']
df_Prom_TATA

,Sequence,Label
0,CTCCAC TCCACT CCACTT CACTTT ACTTTT CTTTTT TTTT...,1
1,CCAGCA CAGCAG AGCAGA GCAGAT CAGATG AGATGG GATG...,1
2,TCAGAG CAGAGA AGAGAA GAGAAA AGAAAC GAAACT AAAC...,1
3,GAAAAG AAAAGG AAAGGT AAGGTA AGGTAA GGTAAA GTAA...,1
4,CAGGGT AGGGTC GGGTCT GGTCTG GTCTGC TCTGCA CTGC...,1
...,...,...
6125,TCGAAT CGAATG GAATGA AATGAA ATGAAC TGAACC GAAC...,0
6126,AGGCCA GGCCAG GCCAGA CCAGAC CAGACC AGACCC GACC...,0
6127,GCTTCA CTTCAG TTCAGG TCAGGG CAGGGC AGGGCT GGGC...,0
6128,AGGCGG GGCGGC GCGGCC CGGCCG GGCCGC GCCGCC CCGC...,0


In [10]:
train, test = train_test_split(df_Prom_TATA, test_size=0.15)
print(train.groupby(['Label']).size(), test.groupby(['Label']).size())
train.to_csv("/data/private/pdutta/DNABERT_data/TATA_Prom-249to50/train.tsv", sep="\t", index=False)
test.to_csv("/data/private/pdutta/DNABERT_data/TATA_Prom-249to50/dev.tsv", sep="\t", index=False)

Label
0    2605
1    2605
dtype: int64 Label
0    460
1    460
dtype: int64


In [11]:
df_coreProm70_TATA['Sequence'] = df_train['CoreProm-35to34'].apply(seq2kmer, args=(6,))
df_coreProm70_TATA['Label'] = df_train['label']
df_coreProm70_TATA

,Sequence,Label
0,ACGGGA CGGGAT GGGATA GGATAA GATAAA ATAAAG TAAA...,1
1,CAGACC AGACCT GACCTA ACCTAT CCTATA CTATAA TATA...,1
2,CTTCCT TTCCTA TCCTAT CCTATA CTATAA TATAAA ATAA...,1
3,TGGTCC GGTCCC GTCCCA TCCCAT CCCATA CCATAA CATA...,1
4,CGCGGG GCGGGC CGGGCA GGGCAT GGCATA GCATAA CATA...,1
...,...,...
6125,TTGGTT TGGTTC GGTTCA GTTCAA TTCAAA TCAAAA CAAA...,0
6126,CACTTA ACTTAC CTTACA TTACAA TACAAA ACAAAT CAAA...,0
6127,TGGGGA GGGGAT GGGATA GGATAT GATATG ATATGA TATG...,0
6128,ATCCTA TCCTAT CCTATA CTATAT TATATT ATATTA TATT...,0


In [12]:
train, test = train_test_split(df_coreProm70_TATA, test_size=0.15)
print(train.groupby(['Label']).size(), test.groupby(['Label']).size())
train.to_csv("/data/private/pdutta/DNABERT_data/TATA_CoreProm-35to34/train.tsv", sep="\t", index=False)
test.to_csv("/data/private/pdutta/DNABERT_data/TATA_CoreProm-35to34/dev.tsv", sep="\t", index=False)

Label
0    2590
1    2620
dtype: int64 Label
0    475
1    445
dtype: int64


In [13]:
df_coreProm90_TATA['Sequence'] = df_train['CoreProm-45to44'].apply(seq2kmer, args=(6,))
df_coreProm90_TATA['Label'] = df_train['label']
df_coreProm90_TATA

,Sequence,Label
0,CGGCAG GGCAGG GCAGGG CAGGGG AGGGGA GGGGAA GGGA...,1
1,CTTCTC TTCTCC TCTCCC CTCCCA TCCCAG CCCAGC CCAG...,1
2,ATCCTC TCCTCT CCTCTC CTCTCT TCTCTG CTCTGC TCTG...,1
3,GGCTGA GCTGAC CTGACA TGACAG GACAGA ACAGAT CAGA...,1
4,CTCCAC TCCACC CCACCG CACCGG ACCGGC CCGGCC CGGC...,1
...,...,...
6125,GTGCTG TGCTGT GCTGTC CTGTCT TGTCTC GTCTCT TCTC...,0
6126,GCACTC CACTCG ACTCGG CTCGGA TCGGAG CGGAGC GGAG...,0
6127,GGTTGG GTTGGG TTGGGA TGGGAG GGGAGG GGAGGT GAGG...,0
6128,GTGCCA TGCCAC GCCACA CCACAA CACAAA ACAAAA CAAA...,0


In [14]:
train, test = train_test_split(df_coreProm90_TATA, test_size=0.15)
print(train.groupby(['Label']).size(), test.groupby(['Label']).size())
train.to_csv("/data/private/pdutta/DNABERT_data/TATA_CoreProm-45to44/train.tsv", sep="\t", index=False)
test.to_csv("/data/private/pdutta/DNABERT_data/TATA_CoreProm-45to44/dev.tsv", sep="\t", index=False)

Label
0    2572
1    2638
dtype: int64 Label
0    493
1    427
dtype: int64
